# A case study of Payment Defaults
A default occurs when a borrower stops making required payment on their debt. As such, they become a liability to the lender and can disrupt their business. Because they are a significant cost factor, they need to be avoided. 
In this case study, I present an analysis of the payment defaults to identify the key trends that can help our client avoid default-prone customers in the future. I organised my case study as follows:
1. Data cleaning
2. Feature engineering
3. Exploratory Data Analysis
4. Predictive modelling
5. Conclusions

For this case study, I have with me two datasets: **clients.csv** and **payments.csv**. Here is an overview of the data I have:

**clients.csv** 

Here each line represents a unique client and their information.
- ClientID : Unique Identifier for the client
- Entity Type :  Business type
- Entity Year Established : First year the business was open

 

**payments.csv**

Here, each line represents a transaction payment for a contract
- TransactionID : Primary key / unique identifier for each transaction
- ContractID : Identifier for the contract. A single contract will have multiple repayments
- ClientID : the client of the contract
- TransactionDate : The date of the transaction in EPOCH format. 
- PaymentAmount : Amount to be repaid
- Payment Code : Categorical variable showing if the payment was made (PAYMENT) or not (DEFAULT)

In this notebook, I will be cleaning the data. In saying so, I will be addressing the following issues within the data:
- Missing values
- Duplicate values
- Invalid Data
- Data formatting and data types



In [55]:
import pandas as pd
import numpy as np

# Section 1: Client data

First I am reading the clients data into `clients` dataframe using `pd.read_csv`

In [56]:
clients = pd.read_csv("data/Clients.csv")
clients.head()

,client_id,entity_type,entity_year_established
0,786,Australian Private Company,2002
1,230,Australian Private Company,2008
2,282,Individual/Sole Trader,2001
3,447,Australian Private Company,2013
4,310,Individual/Sole Trader,2015


In [57]:
clients.shape

(1287, 3)

I find that the clients.csv has 1287 rows and 3 columns.

### 1.1 Check for missing values and data formatting issues

I use `.info()` method to check for any missing values in the `clients` dataframe. 

In [58]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   client_id                1287 non-null   int64 
 1   entity_type              1287 non-null   object
 2   entity_year_established  1287 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 30.3+ KB


I do not find any missing values found in the `client`. However, I notice that `entity_year_established` is of type int. Although I can convert it to `datetime` format, I choose to keep this column as an `int` so that it is easier to work with.

### 1.2 Check for duplicate values
I then check to see if we have any duplicates in the dataframe. 

In [59]:
clients.client_id.nunique()

1281

Earlier I noted that the clients dataset has 1287 rows. However, I only find 1281 unique values. So I probe the data for any duplicates. 

In [60]:
clients[clients.duplicated()]

,client_id,entity_type,entity_year_established


I do not find any duplicates in the `clients` dataframe.

Now, I check for the unique values in each column. 

In [61]:
clients['client_id'].value_counts()

591     3
165     2
473     2
1262    2
797     2
       ..
281     1
122     1
1118    1
181     1
960     1
Name: client_id, Length: 1281, dtype: int64

I notice that some client_id occur multiple times but there are no identical rows in the dataset. So I check what is going on with three client_id, 591, 165 and 473.

In [62]:
clients[clients.client_id.isin([591, 165, 473])].sort_values(by='client_id')

,client_id,entity_type,entity_year_established
401,165,Australian Private Company,2015
1115,165,Individual/Sole Trader,2006
332,473,Australian Private Company,2016
350,473,Individual/Sole Trader,2008
245,591,Australian Private Company,2013
306,591,Australian Private Company,2007
816,591,Australian Private Company,2015


I notice two things in the test sample above. Although the `client_id` is the same, either the `entity_year_established`, `entity_type` or both are different. If there was a way to confirm (e.g., contacting our client, publicly available information about client_id), we can update our clients dataset accordingly. However, I do not have any such way to do so. From my analysis in the next section, I find that I only have payment information from 2017-2018. Under the assumption that a company established in 2000s might change its type and re-establish itself in later years, I remove the duplicate entries by keeping the latest availble `entity_year_established` for each `client_id`. I note here that such a removal will be affecting my results. 

In [63]:
clients.sort_values(by='entity_year_established', inplace=True)
clients.reset_index(drop=True, inplace=True)
clients.drop_duplicates(keep='last', subset='client_id', inplace=True)

In [64]:
clients[clients.client_id.isin([591, 165, 473])].sort_values(by='client_id')

,client_id,entity_type,entity_year_established
1068,165,Australian Private Company,2015
1163,473,Australian Private Company,2016
1112,591,Australian Private Company,2015


In [65]:
clients.shape

(1281, 3)

Now we can see that our clients have 1281 rows, that matches with the number of unique `client_id`s obtained earlier in this subsection. 

I now check for any duplicates in the entity_type, especially the ones with different case or spelling errors. For this, I get the unique values of the entity_types.

In [66]:
list_of_entities = clients.groupby('entity_type').nunique().index.to_list()
list_of_entities

['Australian Private Company',
 'Australian Proprietary Company',
 'Australian Public Company',
 'Discretionary Investment Trust',
 'Discretionary Trading Trust',
 'Family Partnership',
 'Fixed Unit Trust',
 'Hybrid Trust',
 'Individual/Sole Trader',
 'Other Partnership']

In [67]:
len(list_of_entities)

10

Since we only have 10 business types, I can check for case changes and spelling errors by eye. However, I also wrote a program to check for any case changes.

In [68]:
test_list = []
for i in list_of_entities:
    if i.lower() not in test_list:
        test_list.append(i)
    else:
        print(i,end=' ')

I checked for spelling errors by eye. 

### 1.3 Invalid data

In this subsection, I aim to deal with any form of invalid data. I am going to probe the clients dataset for **Entity_year_established > 2018**. We only have payments information till 2018 so an entity established later than 2018 is not valid. 

In [69]:
clients[clients.entity_year_established > 2018]

,client_id,entity_type,entity_year_established


From the above test, I find that there are no businesses established after 2018. 

Now that the clients data is cleaned, I will save the cleaned data to a folder.

In [70]:
clients.to_csv("cleaned_data/clients.csv", index=False)

Now I will clean the payments data in a similar manner.

# Section 2: Payments data

First, I load my payments dataset into `payments`

In [71]:
payments = pd.read_csv("data/Payments.csv")
payments.head()

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code
0,20175,927,1,1527012511,66.66,PAYMENT
1,8485,927,1,1511716095,66.66,PAYMENT
2,13778,927,1,1519319303,66.66,PAYMENT
3,22768,927,1,1529863724,66.66,PAYMENT
4,15698,927,1,1521738504,66.66,PAYMENT


### 2.1 Missing values and data formatting issues

In [72]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25559 entries, 0 to 25558
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    25559 non-null  int64  
 1   contract_id       25559 non-null  int64  
 2   client_id         25559 non-null  int64  
 3   transaction_date  25559 non-null  int64  
 4   payment_amt       25559 non-null  float64
 5   payment_code      25559 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 1.2+ MB


For the dataframe information, we can see that there is no missing data for payments dataset. 

Transaction date is in EPOCH format. I now will convert it to a timestamp. 

In [73]:
payments['transaction_date_dt'] = pd.to_datetime(payments['transaction_date'], unit='s')#.strftime('%Y-%m-%d')
payments.head()

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code,transaction_date_dt
0,20175,927,1,1527012511,66.66,PAYMENT,2018-05-22 18:08:31
1,8485,927,1,1511716095,66.66,PAYMENT,2017-11-26 17:08:15
2,13778,927,1,1519319303,66.66,PAYMENT,2018-02-22 17:08:23
3,22768,927,1,1529863724,66.66,PAYMENT,2018-06-24 18:08:44
4,15698,927,1,1521738504,66.66,PAYMENT,2018-03-22 17:08:24


In [74]:
payments.transaction_date_dt.dt.year.value_counts()

2018    15109
2017    10450
Name: transaction_date_dt, dtype: int64

I find that only the transactions in the years 2017-2018 are reported in the payments data. 

Now I convert the payment_code to a string object.

In [75]:
payments['payment_code'] = payments['payment_code'].astype(str)
payments.head()

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code,transaction_date_dt
0,20175,927,1,1527012511,66.66,PAYMENT,2018-05-22 18:08:31
1,8485,927,1,1511716095,66.66,PAYMENT,2017-11-26 17:08:15
2,13778,927,1,1519319303,66.66,PAYMENT,2018-02-22 17:08:23
3,22768,927,1,1529863724,66.66,PAYMENT,2018-06-24 18:08:44
4,15698,927,1,1521738504,66.66,PAYMENT,2018-03-22 17:08:24


### 2.2 Invalid data

Then I check if the values in payment_code and see if there are any unknown values. 

In [76]:
payments.groupby('payment_code').client_id.count()

payment_code
DEFAULT     2219
PAYMENT    23340
Name: client_id, dtype: int64

I check below if there are any clients in payments who are not in our clients dataset.

In [77]:
payments[~payments['client_id'].isin(clients['client_id'])]

,transaction_id,contract_id,client_id,transaction_date,payment_amt,payment_code,transaction_date_dt


For the numerical columns, I check the statistical summary to find any invalid values. 

In [78]:
payments.describe()

,transaction_id,contract_id,client_id,transaction_date,payment_amt
count,25559.000000,25559.000000,25559.000000,2.555900e+04,25559.000000
mean,12780.000000,758.221409,602.886811,1.517126e+09,1221.455691
std,7378.392101,352.190207,344.782295,9.964331e+06,4346.049363
min,1.000000,1.000000,1.000000,1.499019e+09,-136.660000
25%,6390.500000,510.000000,308.000000,1.508433e+09,93.330000
50%,12780.000000,732.000000,593.000000,1.518110e+09,266.660000
75%,19169.500000,991.000000,894.000000,1.525976e+09,833.330000
max,25559.000000,1643.000000,1281.000000,1.532456e+09,200000.050000


First thing, I notice is the presence of negative values in `payment_amt`. A negative payment amount can mean that our client owes the business. In this case, the client is overpaying so this information does not contribute to payment defaults. We can treat this issue in two ways: 
1. We can set the negative `payment_amt`values to 0 
2. We can drop these rows. 
Setting the `payment_amt` values to zero can affect our future analysis, especially when we compute mean. So, I chose to drop the rows with `payment_amt` < 0. 

In [79]:
payments.drop(payments[payments['payment_amt']<0].index, inplace=True)#payments[payments['payment_amt']==0]

In [80]:
payments.describe()

,transaction_id,contract_id,client_id,transaction_date,payment_amt
count,25557.000000,25557.000000,25557.000000,2.555700e+04,25557.000000
mean,12779.636734,758.241030,602.887193,1.517125e+09,1221.558777
std,7378.559491,352.196648,344.782741,9.964524e+06,4346.203781
min,1.000000,1.000000,1.000000,1.499019e+09,0.010000
25%,6390.000000,510.000000,308.000000,1.508433e+09,93.330000
50%,12779.000000,732.000000,593.000000,1.518110e+09,266.660000
75%,19170.000000,991.000000,894.000000,1.525976e+09,833.330000
max,25559.000000,1643.000000,1281.000000,1.532456e+09,200000.050000


As one can see, we removed the rows with negative payment_amt.

### 2.3 Duplicate values

Now I check for duplicates in the payments data. I will check for duplicates transaction_id. One client may make multiple repayments under the same contract so I expect to find duplicates under client_id and contract_id. However, I expect the transaction_id to be unique. 

In [81]:
payments.shape

(25557, 7)

In [82]:
payments.transaction_id.nunique()

25557

I find that there are 25557 transactions in payments data and each transaction is unique.

In [83]:
payments.client_id.nunique()

1281

I find 1281 unique clients in payments data. This number matches with the number of unique clients from the clients dataset. 

In [84]:
payments.groupby('client_id').transaction_id.count().sort_values()

client_id
134       1
387       1
1281      2
757       2
762       2
       ... 
1128     78
777      79
1124     81
859      92
413     105
Name: transaction_id, Length: 1281, dtype: int64

As per our expectations, we find that the client_ids are not unique. I note that each client makes atleast one transaction. Now, I check the contract_id.

In [85]:
payments.contract_id.nunique()

1643

In [86]:
payments.groupby('contract_id').transaction_id.count().sort_values()

contract_id
4         1
8         1
1643      2
85        2
84        2
       ... 
526      76
701      77
613      79
649      84
866     105
Name: transaction_id, Length: 1643, dtype: int64

There are 1643 unique contracts in our dataset. Similar to client_id, each contract_id had atleast one transaction.

### 2.4 Dropping some columns from payments

To keep things compact and clear, it is useful to drop columns which do not add unnecessary information. For my analysis, I will not need `transaction_date`. So I will drop this column. The information from `transaction_date` has been copied to a datatime format in `transaction_date_dt` so I can drop the less readable column. 

In [87]:
payments.drop(columns=['transaction_date'], inplace=True)

I will also rename the `transaction_date_dt` column to `transaction_date` for simplicity.

In [88]:
payments.rename(columns={'transaction_date_dt':'transaction_date'}, inplace=True)
payments.columns

Index(['transaction_id', 'contract_id', 'client_id', 'payment_amt',
       'payment_code', 'transaction_date'],
      dtype='object')

Now I will write my cleaned payments data to a file in case I need to access them later.

In [89]:
payments.to_csv("cleaned_data/payments.csv", index=False)

# Section 3: Combine the clients and payments data

Now that the payments and clients datasets are clean, we can combine them into one single dataframe. We can then use this data to perform exploratory data analysis. 

In [90]:
df = pd.merge(clients, payments, on='client_id', how='inner')
df.head()

,client_id,entity_type,entity_year_established,transaction_id,contract_id,payment_amt,payment_code,transaction_date
0,240,Australian Private Company,1999,23556,1557,50.00,PAYMENT,2018-07-03 18:08:45
1,240,Australian Private Company,1999,23997,1557,1716.67,PAYMENT,2018-07-08 18:09:22
2,240,Australian Private Company,1999,23998,1557,64.99,PAYMENT,2018-07-08 18:09:22
3,240,Australian Private Company,1999,23555,1557,20050.04,PAYMENT,2018-07-03 18:08:45
4,11,Australian Private Company,1999,5009,220,199.99,PAYMENT,2017-09-24 18:08:12


The result for doing an inner or outer join was the same. So I chose to go along with a inner join. 

In [91]:
print(payments.shape, clients.shape, df.shape)

(25557, 6) (1281, 3) (25557, 8)


I checked that the combined dataframe `df` has same number of rows as `payments` with two new columns from `clients` added. 

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25557 entries, 0 to 25556
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   client_id                25557 non-null  int64         
 1   entity_type              25557 non-null  object        
 2   entity_year_established  25557 non-null  int64         
 3   transaction_id           25557 non-null  int64         
 4   contract_id              25557 non-null  int64         
 5   payment_amt              25557 non-null  float64       
 6   payment_code             25557 non-null  object        
 7   transaction_date         25557 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 1.8+ MB


Further, I checked for any missing values in `df` and I did not find one. Lastly, I check for any duplicate transactions.

In [93]:
df[df.duplicated(subset=['transaction_id'])]

,client_id,entity_type,entity_year_established,transaction_id,contract_id,payment_amt,payment_code,transaction_date


In [94]:

df.to_csv("cleaned_data/combined_data.csv", index=False)

# Section 4: Feature engineering

To the data I have, I want to introduce two new features:
1. `transaction_month`: Month of the transaction
2. `transaction_year` : Year of the transaction
3. `years_from_establishment`: The years from establishment of the business to transaction date.

In [95]:
new_ft = df.copy()

### 4.1 Transaction month and year
Using the datetime properties, one can easily obtain the month and year of the transaction from `transaction_date`


In [96]:
new_ft['transaction_year'] = new_ft.transaction_date.dt.year
new_ft.head()

,client_id,entity_type,entity_year_established,transaction_id,contract_id,payment_amt,payment_code,transaction_date,transaction_year
0,240,Australian Private Company,1999,23556,1557,50.00,PAYMENT,2018-07-03 18:08:45,2018
1,240,Australian Private Company,1999,23997,1557,1716.67,PAYMENT,2018-07-08 18:09:22,2018
2,240,Australian Private Company,1999,23998,1557,64.99,PAYMENT,2018-07-08 18:09:22,2018
3,240,Australian Private Company,1999,23555,1557,20050.04,PAYMENT,2018-07-03 18:08:45,2018
4,11,Australian Private Company,1999,5009,220,199.99,PAYMENT,2017-09-24 18:08:12,2017


I wanted the transaction_month to be of the format "MM-YYYY" so I used the following method: 

In [97]:
new_ft['transaction_month'] = new_ft['transaction_date'].dt.to_period('M')
new_ft.head()

,client_id,entity_type,entity_year_established,transaction_id,contract_id,payment_amt,payment_code,transaction_date,transaction_year,transaction_month
0,240,Australian Private Company,1999,23556,1557,50.00,PAYMENT,2018-07-03 18:08:45,2018,2018-07
1,240,Australian Private Company,1999,23997,1557,1716.67,PAYMENT,2018-07-08 18:09:22,2018,2018-07
2,240,Australian Private Company,1999,23998,1557,64.99,PAYMENT,2018-07-08 18:09:22,2018,2018-07
3,240,Australian Private Company,1999,23555,1557,20050.04,PAYMENT,2018-07-03 18:08:45,2018,2018-07
4,11,Australian Private Company,1999,5009,220,199.99,PAYMENT,2017-09-24 18:08:12,2017,2017-09


I checked by eye that the transaction_month and transaction_year are accurate for a sample of five transactions. I then copy the final dataframe with all the new features to a new file.

### Years from establishment

In [98]:
new_ft['years_from_establishment'] = new_ft['transaction_year'] - new_ft['entity_year_established']

In [99]:
new_ft.to_csv("generated_data/final_df.csv", index=False)